In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import imp

sn.set_context('notebook')

# Estimating loads in unmonitored regions - 2017

The new model can be used to estimate loads in unmonitored areas. We know the regine ID for each of the 155 stations where water chemistry is measured, and we also know which OSPAR region each monitoring site drains to. We want to use observed data to estimate loads upstream of each monitoring point, and modelled data elsewhere. This can be achieved using the output from the new model.

This notebook is based on the one [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/loads_unmonitored_regions.ipynb). It first runs the NOPE model for 2017 and then extracts data for unmonitored regions.

In [2]:
# Import model
nope_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
             r'\Python\rid\notebooks\nope.py')
nope = imp.load_source('nope', nope_path)

# Connect to db
resa2_basic_path = (r'C:\Data\James_Work\Staff\Heleen_d_W\ICP_Waters\Upload_Template'
                    r'\useful_resa2_code.py')
resa2_basic = imp.load_source('useful_resa2_code', resa2_basic_path)
engine, conn = resa2_basic.connect_to_resa2()

## 1. Generate model input file

In [3]:
# Year of interest
year = 2017

# Parameters of interest
par_list = ['Tot-N', 'Tot-P']

# Folder containing NOPE data
nope_fold = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
             r'\NOPE\NOPE_Core_Input_Data')

# Ouput path for model file
out_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\NOPE\NOPE_Annual_Inputs\nope_input_data_%s.csv' % year)

In [4]:
# Make input file
df = nope.make_rid_input_file(year, engine, nope_fold, out_csv,
                              par_list=par_list)

C:\Data\Anaconda2\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


## 2. Run model

In [5]:
%%time
# Input file
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\NOPE\NOPE_Annual_Inputs\nope_input_data_%s.csv' % year)

# Run model
g = nope.run_nope(in_csv, par_list)

Wall time: 7.57 s


## 3. Save results

In [6]:
# Save results as csv
out_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\NOPE\nope_results_%s.csv' % year)
df = nope.model_to_dataframe(g, out_path=out_csv)

df.head()

,regine,regine_ned,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,local_q_reg_m3/s,local_ren_tot-n_tonnes,local_ren_tot-p_tonnes,local_runoff_mm/yr,local_spr_tot-n_tonnes,local_spr_tot-p_tonnes,local_trans_tot-n,local_trans_tot-p,local_urban_tot-n_tonnes,local_urban_tot-p_tonnes
0,001.222Z,001.2220,0.952241,0.054522,0.108471,0.007092,1.985979,0.073503,0.952241,0.054522,...,0.049509,0.0,0.0,251.418992,0.097407,0.006197,1.00,1.00,0.000000,0.000000
1,002.DGBZ,002.DGB0,0.000000,0.000000,0.000000,0.000000,6.020845,0.128697,0.000000,0.000000,...,0.926409,0.0,0.0,339.475071,0.000000,0.000000,0.97,0.83,0.000000,0.000000
2,123.A1Z,123.A12,8.779892,0.122196,0.488847,0.023534,16.881886,0.312083,11.099844,0.244945,...,0.824476,0.0,0.0,652.463681,0.449887,0.064798,0.81,0.30,2.864138,0.409163
3,212.FAC,212.FAB0,0.000000,0.000000,0.021858,0.000787,7.364415,0.074980,0.000000,0.000000,...,0.548659,0.0,0.0,318.294803,0.023009,0.001036,0.95,0.76,0.000000,0.000000
4,135.1AC,135.1AB,0.000000,0.000000,0.000000,0.000000,5.573545,0.057673,0.000000,0.000000,...,1.045270,0.0,0.0,2346.166868,0.000000,0.000000,0.85,0.42,0.000000,0.000000


In [7]:
# Save version with main catchments only
main_list = ["%03d." % i for i in range(1, 316)]
df2 = df.query('regine in @main_list')
df2.sort_values('regine', inplace=True)

# Save
out_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\NOPE\nope_results_%s_main_catchs.csv' % year)
df2.to_csv(out_csv, index=False, encoding='utf-8')

C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## 4. Explore results

### 4.1. Total N and P

####  4.1.1. Identify areas with monitoring data

Where observations are available, we want to use them in preference to the model output. This means identifying all the catchments with observed data and substracting the model results for these locations. This is more complicated than it appears, because a small number of observed catchments are upstream of others, so subtracting all the loads for the 155 monitored catchments involves "double accounting", which we want to avoid. The first step is therefore to identify the downstream-most nodes for the monitored areas i.e. for the cases where one catchment is upstream of another, we just want the downstream node.

In [8]:
# Read station data
in_xlsx = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Data\RID_Sites_List.xlsx')
stn_df = pd.read_excel(in_xlsx, sheet_name='RID_All')

# Get just cols of interest and drop duplicates 
# (some sites are in the same regine)
stn_df = stn_df[['ospar_region', 'nve_vassdrag_nr']].drop_duplicates()

# Get catch IDs with calib data
calib_nds = set(stn_df['nve_vassdrag_nr'].values)

# Build network
in_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\NOPE\NOPE_Annual_Inputs\nope_input_data_1990.csv')
g, nd_list = nope.build_calib_network(in_path, calib_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df['nve_vassdrag_nr'].isin(ds_nds)]

#### 4.1.2. Sum model results for monitored locations

In [9]:
# Read model output
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\NOPE\nope_results_%s.csv' % year)
nope_df = pd.read_csv(in_csv)

# Join accumulated outputs to stns of interest
mon_df = pd.merge(stn_df, nope_df, how='left',
                  left_on='nve_vassdrag_nr',
                  right_on='regine')

# Groupby OSPAR region
mon_df = mon_df.groupby('ospar_region').sum()

# Get just accum cols
cols = [i for i in mon_df.columns if i.split('_')[0]=='accum']
mon_df = mon_df[cols]

mon_df.head()

,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,accum_aqu_tot-n_tonnes,accum_aqu_tot-p_tonnes,accum_ind_tot-n_tonnes,accum_ind_tot-p_tonnes,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2
ospar_region,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,82.443341,7.834522,5176.105493,74.571260,140.062687,5.306710,0.0,0.0,0.000000,0.000000,4953.599465,61.430029,1092.431980,31.051726,3.222816,48.852061,4.393329,63555.61
NORTH SEA,398.114032,34.709115,14311.445332,198.767254,2576.926355,67.873391,0.0,0.0,14.604647,3.239419,11336.404945,96.184748,1817.153129,221.178990,19.118282,127.468243,10.174945,23353.19
NORWEGIAN SEA2,775.021255,58.041882,14844.550054,289.745057,2691.201822,73.822372,0.0,0.0,234.065300,20.538000,11378.326978,157.880804,2193.065293,308.740698,14.133349,191.451437,20.308988,45896.63
SKAGERAK,3834.769077,104.293625,31676.461657,482.927139,11190.254228,234.760312,0.0,0.0,174.750659,25.376397,16651.438352,143.873202,2277.912358,2697.904329,22.805807,860.034463,50.564597,93945.27


This table gives the **modelled** inputs to each OSPAR region from catchments for which we have observed data. We want to subtract these values from the overall modelled inputs to each region and substitute the observed data instead.

The trickiest part of this is that the OSPAR regions in the TEOTIL catchment network (and therefore the network for my new model too) don't exactly match the new OSPAR definitions. The OSPAR boundaries were updated relatively recently, so instead of simply selecting the desired OSPAR region in the model output, I need to aggregate based on vassdragsnummers.

**Note:** Eventually, it would be a good idea to update the network information in `regine.csv` to reflect the current OSPAR regions.

#### 4.1.3. Group model output according to "new" OSPAR regions

In [10]:
# Define "new" OSPAR regions
os_dict = {'SKAGERAK':(1, 23),
           'NORTH SEA':(24, 90),
           'NORWEGIAN SEA2':(91, 170),
           'LOFOTEN-BARENTS SEA':(171, 247)}

# Container for results
df_list = []

# Loop over model output
for reg in os_dict.keys():
    min_id, max_id = os_dict[reg]
    
    regs = ['%03d.' % i for i in range(min_id, max_id+1)]
    
    # Get data for this region
    df2 = nope_df[nope_df['regine'].isin(regs)]
    
    # Get just accum cols
    cols = [i for i in df2.columns if i.split('_')[0]=='accum']
    df2 = df2[cols]
    
    # Add region
    df2['ospar_region'] = reg
    
    # Add sum to output
    df_list.append(df2)

# Build df
os_df = pd.concat(df_list, axis=0)

# Aggregate
os_df = os_df.groupby('ospar_region').sum()

os_df.head()

,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,accum_aqu_tot-n_tonnes,accum_aqu_tot-p_tonnes,accum_ind_tot-n_tonnes,accum_ind_tot-p_tonnes,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2
ospar_region,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,17190.483170,2908.092753,28642.698118,3099.657383,595.572645,29.175909,15813.914120,2730.804920,81.010000,5.899000,10856.642304,162.388721,2699.458709,973.503273,132.706985,311.490663,37.796028,138090.89
NORTH SEA,23844.304274,3846.524371,53744.066879,4263.536606,6228.068783,191.955980,19136.870065,3299.400686,543.602497,79.994211,23671.693822,225.056255,3841.432106,3393.530641,380.365147,689.977145,80.934792,59314.38
NORWEGIAN SEA2,29463.273490,4886.614885,57730.276597,5423.202191,7337.978868,233.537202,24982.000127,4314.385004,1144.603800,178.415500,20929.024239,303.050104,4312.521919,2519.615155,298.695474,715.345793,87.036705,113934.05
SKAGERAK,11121.896691,287.897431,42022.621490,754.961419,12973.642753,309.911458,29.494102,5.217279,1125.258659,74.905797,17927.082046,157.152530,2400.772577,8833.046005,134.070338,1021.112201,67.351677,102574.69


We can now calculate the unmonitored component by simply subtracting the values modelled upstream of monitoring stations from the overall modelled inputs to each OSPAR region.

#### 4.1.4. Estimate loads in unmonitored areas

In [11]:
# Calc unmonitored loads
unmon_df = os_df - mon_df

# Write output
out_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\NOPE\unmonitored_loads_%s.csv' % year)
unmon_df.to_csv(out_csv, encoding='utf-8', index_label='ospar_region')

unmon_df.round(0).astype(int).T

ospar_region,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
accum_all_point_tot-n_tonnes,17108,23446,28688,7287
accum_all_point_tot-p_tonnes,2900,3812,4829,184
accum_all_sources_tot-n_tonnes,23467,39433,42886,10346
accum_all_sources_tot-p_tonnes,3025,4065,5133,272
accum_anth_diff_tot-n_tonnes,456,3651,4647,1783
accum_anth_diff_tot-p_tonnes,24,124,160,75
accum_aqu_tot-n_tonnes,15814,19137,24982,29
accum_aqu_tot-p_tonnes,2731,3299,4314,5
accum_ind_tot-n_tonnes,81,529,911,951
accum_ind_tot-p_tonnes,6,77,158,50


#### 4.1.5. Aggregate values to required quantities

In [12]:
# Aggregate to match report
unmon_df['flow'] = unmon_df['accum_q_m3/s']*60*60*24/1000. # 1000s m3/day

unmon_df['sew_n'] = unmon_df['accum_ren_tot-n_tonnes'] + unmon_df['accum_spr_tot-n_tonnes']
unmon_df['sew_p'] = unmon_df['accum_ren_tot-p_tonnes'] + unmon_df['accum_spr_tot-p_tonnes']

unmon_df['ind_n'] = unmon_df['accum_ind_tot-n_tonnes']
unmon_df['ind_p'] = unmon_df['accum_ind_tot-p_tonnes']

unmon_df['fish_n'] = unmon_df['accum_aqu_tot-n_tonnes']
unmon_df['fish_p'] = unmon_df['accum_aqu_tot-p_tonnes']

unmon_df['diff_n'] = unmon_df['accum_anth_diff_tot-n_tonnes'] + unmon_df['accum_nat_diff_tot-n_tonnes']
unmon_df['diff_p'] = unmon_df['accum_anth_diff_tot-p_tonnes'] + unmon_df['accum_nat_diff_tot-p_tonnes']

new_df = unmon_df[['flow', 'sew_n', 'sew_p', 
                   'ind_n', 'ind_p', 'fish_n', 
                   'fish_p', 'diff_n', 'diff_p']]

# Total for Norway
new_df.loc['NORWAY'] = new_df.sum(axis=0)

# Reorder rows
new_df = new_df.reindex(['NORWAY', 'LOFOTEN-BARENTS SEA', 'NORTH SEA', 
                         'NORWEGIAN SEA2', 'SKAGERAK'])

new_df.round().astype(int)

C:\Data\Anaconda2\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p
ospar_region,,,,,,,,,
NORWAY,507481,13971,1074,2471,290,59962,10350,39601,771
LOFOTEN-BARENTS SEA,138847,1205,163,81,6,15814,2731,6359,125
NORTH SEA,174898,3735,432,529,77,19137,3299,15986,253
NORWEGIAN SEA2,183121,2735,351,911,158,24982,4314,14197,305
SKAGERAK,10615,6296,128,951,50,29,5,3059,88


## 5. Other N and P species

Tore's procedure `RESA2.FIXTEOTILPN` defines simple correction factors for estimating PO4, NO3 and NH4 from total P and N. The table below lists the factors used.

|   Source    | Phosphate | Nitrate | Ammonium |
|:-----------:|:---------:|:-------:|:--------:|
|    Sewage   |     0.600 |   0.050 |    0.750 |
|   Industry  |     0.600 |   0.050 |    0.750 |
| Aquaculture |     0.690 |   0.110 |    0.800 |
|   Diffuse   |     0.246 |   0.625 |    0.055 |


In [13]:
# Dict of conversion factors
con_dict = {('sew', 'po4'):('p', 0.6),
            ('ind', 'po4'):('p', 0.6),
            ('fish', 'po4'):('p', 0.69),
            ('diff', 'po4'):('p', 0.246),
            ('sew', 'no3'):('n', 0.05),
            ('ind', 'no3'):('n', 0.05),
            ('fish', 'no3'):('n', 0.11),
            ('diff', 'no3'):('n', 0.625),
            ('sew', 'nh4'):('n', 0.75),
            ('ind', 'nh4'):('n', 0.75),
            ('fish', 'nh4'):('n', 0.8),
            ('diff', 'nh4'):('n', 0.055)}

# Apply factors
for src in ['sew', 'ind', 'fish', 'diff']:
    for spc in ['po4', 'no3', 'nh4']:
        el, fac = con_dict[(src, spc)]
        new_df[src+'_'+spc] = fac * new_df[src+'_'+el]
        
new_df.round().astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,507481,138847,174898,183121,10615
sew_n,13971,1205,3735,2735,6296
sew_p,1074,163,432,351,128
ind_n,2471,81,529,911,951
ind_p,290,6,77,158,50
fish_n,59962,15814,19137,24982,29
fish_p,10350,2731,3299,4314,5
diff_n,39601,6359,15986,14197,3059
diff_p,771,125,253,305,88
sew_po4,645,98,259,211,77


## 6. Other quantities

The model currently only considers N and P, but the project focuses on a wider range of parameters. For now, we simply assume that all measured inputs (`renseanlegg`, `industri` and `akvakultur`) for regines outside of catchments with measured data make it to the sea.

We only want data for catchments that are not monitored i.e. for regine IDs **not** in the graph created above.

In [58]:
# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = ("SELECT SUBSTR(a.regine, 1, 3) AS vassdrag, "
       "  a.type, "
       "  b.name, "
       "  b.unit, "
       "  SUM(c.value * d.factor) as value "
       "FROM RESA2.RID_PUNKTKILDER a, "
       "RESA2.RID_PUNKTKILDER_OUTPAR_DEF b, "
       "RESA2.RID_PUNKTKILDER_INPAR_VALUES c, "
       "RESA2.RID_PUNKTKILDER_INP_OUTP d "
       "WHERE a.anlegg_nr = c.anlegg_nr "
       "AND (1, a.regine) NOT IN %s "
       "AND d.in_pid = c.inp_par_id "
       "AND d.out_pid = b.out_pid "
       "AND c.year = %s "
       "GROUP BY SUBSTR(a.regine, 1, 3), a.type, b.name, b.unit "
       "ORDER BY SUBSTR(a.regine, 1, 3), a.type" % (tuple(nd_list_hack), year))

df = pd.read_sql(sql, engine)

# Tidy
df['par'] = df['type'] + '_' + df['name'] + '_' + df['unit']
del df['name'], df['unit'], df['type']

# Pivot
df = df.pivot(index='vassdrag', columns='par', values='value')
df.reset_index(inplace=True)

In [59]:
def f(x):
    try:
        a = int(x)
        return a
    except:
        return -999

# Convert vassdrag to numbers
df['vass'] = df['vassdrag'].apply(f)

# Get just the main catchments
df = df.query('vass != -999')

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PCB_tonn,...,RENSEANLEGG_Hg_tonn,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass
0,001,0.060180,0.007080,0.010600,0.051710,0.000000e+00,NaN,NaN,0.072700,NaN,...,NaN,NaN,NaN,NaN,NaN,0.80415,140.02585,14.96525,NaN,1
1,002,0.012232,0.005785,0.631446,3.550247,6.038000e-03,NaN,NaN,0.361101,NaN,...,0.001011,0.211513,0.000552,0.000017,0.016230,600.04694,705.61349,10.81220,1.050888,2
2,003,0.000066,0.000012,0.000239,0.002829,7.000000e-07,NaN,NaN,0.000517,NaN,...,0.000022,0.018978,0.000026,0.000021,0.001338,152.30169,293.51446,3.10258,0.112204,3
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,163.09282,1.07263,NaN,4
4,005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,41.49898,0.40689,NaN,5


In [60]:
def f2(x):   
    if x in range(1, 24):
        return 'SKAGERAK'
    elif x in range(24, 91):
        return 'NORTH SEA'
    elif x in range(91, 171):
        return 'NORWEGIAN SEA2'
    elif x in range(171, 248):
        return 'LOFOTEN-BARENTS SEA'
    else:
        return np.nan

# Assign main catchments to OSPAR regions
df['osp_reg'] = df['vass'].apply(f2)

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PCB_tonn,...,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass,osp_reg
0,001,0.060180,0.007080,0.010600,0.051710,0.000000e+00,NaN,NaN,0.072700,NaN,...,NaN,NaN,NaN,NaN,0.80415,140.02585,14.96525,NaN,1,SKAGERAK
1,002,0.012232,0.005785,0.631446,3.550247,6.038000e-03,NaN,NaN,0.361101,NaN,...,0.211513,0.000552,0.000017,0.016230,600.04694,705.61349,10.81220,1.050888,2,SKAGERAK
2,003,0.000066,0.000012,0.000239,0.002829,7.000000e-07,NaN,NaN,0.000517,NaN,...,0.018978,0.000026,0.000021,0.001338,152.30169,293.51446,3.10258,0.112204,3,SKAGERAK
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,163.09282,1.07263,NaN,4,SKAGERAK
4,005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,41.49898,0.40689,NaN,5,SKAGERAK


In [61]:
# Group by OSPAR region
df.fillna(0, inplace=True)
df = df.groupby('osp_reg').sum()
df.drop(0, inplace=True)

# Total for Norway
df.loc['NORWAY'] = df.sum(axis=0)

# Join to model results 
df = new_df.join(df)

# Get cols of interest
umod_cols = ['S.P.M.', 'TOC', 'As', 'Pb', 'Cd', 'Cu', 'Zn', 'Ni', 'Cr', 'Hg']
umod_cols = ['%s_%s_tonn' % (i, j) for i in ['INDUSTRI', 'RENSEANLEGG'] for j in umod_cols]
cols = list(new_df.columns) + umod_cols
cols.remove('RENSEANLEGG_TOC_tonn')
df = df[cols]

df.round(0).astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,507481,138847,174898,183121,10615
sew_n,13971,1205,3735,2735,6296
sew_p,1074,163,432,351,128
ind_n,2471,81,529,911,951
ind_p,290,6,77,158,50
fish_n,59962,15814,19137,24982,29
fish_p,10350,2731,3299,4314,5
diff_n,39601,6359,15986,14197,3059
diff_p,771,125,253,305,88
sew_po4,645,98,259,211,77


## 7. Fish farm copper

Finally, we need to add in the Cu totals from fish farms. The method is similar to that used above, but simpler because we're only interested in one parameter.

In [19]:
# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = ("SELECT SUBSTR(a.regine, 1, 3) AS vassdrag, "
       "  SUM(b.value) as value "
       "FROM RESA2.RID_KILDER_AQUAKULTUR a, "
       "RESA2.RID_KILDER_AQKULT_VALUES b "
       "WHERE a.nr = b.anlegg_nr "
       "AND (1, a.regine) NOT IN %s "
       "AND b.inp_par_id = 41 "
       "AND b.ar = %s "
       "GROUP BY SUBSTR(a.regine, 1, 3), b.inp_par_id "
       "ORDER BY SUBSTR(a.regine, 1, 3), b.inp_par_id" % (tuple(nd_list_hack), year))

aq_df = pd.read_sql(sql, engine)

# Get vassdrag
aq_df['vass'] = aq_df['vassdrag'].apply(f)
aq_df = aq_df.query('vass != -999')

# Calc OSPAR region and group
aq_df['osp_reg'] = aq_df['vass'].apply(f2)
aq_df.fillna(0, inplace=True)
aq_df = aq_df.groupby('osp_reg').sum()
del aq_df['vass']

# Total for Norway
aq_df.loc['NORWAY'] = aq_df.sum(axis=0)

# Rename
aq_df.columns = ['AQUAKULTUR_Cu_tonn',]

# Join model results 
df = df.join(aq_df)

df.round(0).astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,507481,138847,174898,183121,10615
sew_n,13971,1205,3735,2735,6296
sew_p,1074,163,432,351,128
ind_n,2471,81,529,911,951
ind_p,290,6,77,158,50
fish_n,59962,15814,19137,24982,29
fish_p,10350,2731,3299,4314,5
diff_n,39601,6359,15986,14197,3059
diff_p,771,125,253,305,88
sew_po4,645,98,259,211,77


In [20]:
# Write output
out_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Results\Unmon_loads\unmon_loads_%s.csv' % year)
df.to_csv(out_csv)

This data can then be used to create Table 3 in the report - see [this notebook](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/word_data_tables.ipynb) for details.